In [7]:
import os
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy import stats
import matplotlib.pyplot as plt
from tesser import util
from tesser import network
from tesser import rsa
plt.rcParams['figure.figsize'] = [10, 10]
import sys
np.set_printoptions(threshold=sys.maxsize)

In [8]:
###########
#DIRECTORY#
###########
#need to change path depending on whether you're using laptop or desktop:
#desktop:
#data_dir = '/Users/athula/Dropbox/Experiments/TesserScan/neural_results_2020/rsa_allevents'
#event_dir = '/Users/athula/Dropbox/Experiments/TesserScan/Neural_results_2020/rsa_allevents_info'

#laptop:
data_dir = '/Users/athulapudhiyidath/Dropbox/Experiments/tesser_successor/Data/item_betas'
event_dir = '/Users/athulapudhiyidath/Dropbox/Experiments/tesser_successor/TesserScan/rsa_allevents_info'

In [9]:
################
#SUBJECT + ROIs#
################

#load subject data:
subjs = util.subj_list()
subjs_np = np.array(subjs)

#load txt files by subj + ROI:
rois = ['b_hip_ant', 'b_hip_body', 'b_hip_tail', 'b_hip', 'l_hip_ant', 'l_hip_body', 'l_hip_tail', 'l_hip', 'r_hip_ant', 'r_hip_body', 'r_hip_tail', 'r_hip']
rois_np = np.repeat(rois,2)

#conditions
conditions = ('within', 'across')
conditions_np = np.array(conditions*len(rois))
conditions_array = [rois_np, conditions_np]

In [11]:
within_across_roi_overall_subj = np.array([])
within_across_roi_overall_mean = np.array([])
for r in range(len(rois)): 
    #loop through rois 
    roi = rois[r]
    
    within = []
    across = []
    for s in range(len(subjs)):
        #loop through subjects    
        sub = subjs[s]
        
        #getting RSA for a subject + their ROI
        this_auto_rsa = rsa.load_rsa(data_dir, sub, roi)

        #now excluding auto-correlated trials from the diagonal:
        #this_exclude_rsa = exclude_rsa(this_auto_rsa, 5)
            
        #getting the community membership of each item across all runs 
        #getting the run numbers of each item
        #getting the scrambled numbers of each item
        run_all_items = []
        run_all_runs = []
        run_all_seq = []
        for run in list(range(1, 7)):
            this_run_info = util.load_struct_run_info(event_dir, sub, run)
            this_run_item = this_run_info.item
            run_all_items.extend(this_run_item)
    
            run_num = [run]*len(this_run_info)
            run_all_runs.extend(run_num)
    
            run_seq = this_run_info.seq_type
            run_all_seq.extend(run_seq)    

        run_all_comms = []
        for i in range(len(run_all_items)):
            item  = run_all_items[i]
            node_info = network.temp_node_info()
            if item != 0:
                item_comm = node_info.comm[item]
            else:
                item_comm = 'NaN'
        
            run_all_comms.append(item_comm)

        scram_all_seq = []
        nonscram_all_seq = []
        for i in range(len(run_all_seq)):
            seq  = run_all_seq[i]
            if seq == 1:
                scram = 0
                nonscram = 1
            elif seq == 2:
                scram = 1
                nonscram = 0 
            else:
                scram = 'NaN'
                nonscram = 'NaN'
            scram_all_seq.append(scram)
            nonscram_all_seq.append(nonscram)

        nan_all_seq = []
        for i in range(len(run_all_items)):
            obj  = run_all_items[i]
            if obj == 0:
                this_nan = 'NaN'
            else:
                this_nan = 0
            nan_all_seq.append(this_nan)
    
        run_all_comms = np.array(run_all_comms)
        run_all_runs = np.array(run_all_runs)
        run_all_scram = np.array(scram_all_seq)
        run_all_nonscram = np.array(nonscram_all_seq)
        run_all_nans = np.array(nan_all_seq)
        run_all_items = np.array(run_all_items)
        
        #assigning the 906 x 906 matrix with identifiers that indicate: 
        #within community = true 
        #across community = false
        within_comm = rsa.pair_eq(run_all_comms)
        within_comm.shape
        within_comm

        #assigning the 906 x 906 matrix with identifiers that indicate: 
        #within run = false 
        #across community = true
        across_run = rsa.pair_neq(run_all_runs)
        across_run.shape
        across_run

        #assigning the 906 x 906 matrix with identifiers that indicate: 
        #scram = true 
        #non-scram = false
        scram_run = rsa.pair_eq(run_all_scram)
        scram_run.shape
        scram_run

        #assigning the 906 x 906 matrix with identifiers that indicate: 
        #NaN item = 1 
        #other item = 0
        nan_run = np.zeros((len(run_all_nans), len(run_all_nans)))
        exclude = np.where(run_all_nans == 'NaN')
        nan_run[:, exclude] = 1
        nan_run[exclude, :] = 1
        
        #assigning the 906 x 906 matrix with identifiers that indicate: 
        #same object = true 
        #different object = false
        same_object = rsa.pair_eq(run_all_items)
        #same_object.shape
        #same_object  

        within_comm_sim = this_auto_rsa[(within_comm == True) & (across_run == True) & (scram_run == False) & (nan_run == 0) & (same_object == False)]
        across_comm_sim = this_auto_rsa[(within_comm == False) & (across_run == True) & (scram_run == False) & (nan_run == 0) & (same_object == False)]

        within_comm_sim_mean = np.nanmean(within_comm_sim)
        across_comm_sim_mean = np.nanmean(across_comm_sim)
        
        within.append(within_comm_sim_mean)
        across.append(across_comm_sim_mean)

    #by ROI
    within_roi_mean = np.mean(within)
    across_roi_mean = np.mean(across)
    
    within_across_roi_mean = np.column_stack([within_roi_mean, across_roi_mean])
    within_across_roi_all = np.column_stack([within, across])
    
    if r == 0:
        within_across_roi_overall_mean =  within_across_roi_mean
        within_across_roi_mean_tog_all = within_across_roi_all
    else:
        within_across_roi_overall_mean = np.concatenate([within_across_roi_overall_mean, within_across_roi_mean])
        within_across_roi_mean_tog_all = np.concatenate([within_across_roi_mean_tog_all, within_across_roi_all], axis=1)

In [12]:
within_across_roi_overall_mean

array([[0.00071491, 0.00088663],
       [0.00187872, 0.0020139 ],
       [0.00276447, 0.00243689],
       [0.00234623, 0.00239457],
       [0.0007979 , 0.00069943],
       [0.00172357, 0.00175384],
       [0.00234047, 0.00212748],
       [0.00197343, 0.00191543],
       [0.00073738, 0.00113565],
       [0.00227101, 0.00224719],
       [0.00274765, 0.00259394],
       [0.00270896, 0.00279069]])

In [13]:
#MEAN ROI + WITHIN/ACROSS 
roi_mean_sim_df = pd.DataFrame(within_across_roi_overall_mean,
                  index=[rois],
                  columns=['within', 'across'])

roi_mean_sim_df

,within,across
b_hip_ant,0.000715,0.000887
b_hip_body,0.001879,0.002014
b_hip_tail,0.002764,0.002437
b_hip,0.002346,0.002395
l_hip_ant,0.000798,0.000699
l_hip_body,0.001724,0.001754
l_hip_tail,0.002340,0.002127
l_hip,0.001973,0.001915
r_hip_ant,0.000737,0.001136
r_hip_body,0.002271,0.002247


In [14]:
roi_mean_sim_df.to_csv('singletrial_structured_roi_WithinAcross_Overall.csv', index=True)  

In [15]:
within_across_roi_mean_tog_all

array([[ 2.29728971e-03,  2.04193494e-03,  8.99587602e-04,
        -9.00189906e-05,  5.23876990e-03,  4.30350855e-03,
         2.77583934e-03,  1.96704570e-03,  2.37586966e-03,
         1.45593163e-03,  1.69299745e-03,  2.56936044e-04,
         6.58154266e-03,  6.12993504e-03,  3.27085585e-03,
         2.12294100e-03,  3.07047790e-03,  2.93776451e-03,
         1.96087238e-04,  1.59937819e-05,  1.78592080e-03,
         1.37017709e-03,  2.40026586e-03,  1.84383698e-03],
       [ 9.81177353e-04,  1.22352156e-03,  1.91816856e-03,
         2.47439739e-03,  3.85491663e-03,  2.20856088e-03,
         2.48007380e-03,  1.99803407e-03,  2.24286023e-03,
         1.66639702e-03,  1.06224919e-03,  2.00951915e-03,
         3.62239998e-03,  1.17832331e-03,  3.09006692e-03,
         2.39743937e-03,  4.26597866e-04,  1.12416291e-03,
         3.27845615e-03,  3.15642143e-03,  1.52586223e-03,
         1.16837582e-03,  1.94021093e-03,  1.38637879e-03],
       [-2.29513777e-04,  4.67784863e-04,  2.00920036e

In [16]:
#MEAN ROI + WITHIN/ACROSS x SUBJECT:
roi_mean_subj_sim_df = pd.DataFrame(within_across_roi_mean_tog_all, index=[subjs_np], columns=conditions_array)
roi_mean_subj_sim_df

b_hip_ant           b_hip_body           b_hip_tail               b_hip  \
       within    across     within    across     within    across    within   
100  0.002297  0.002042   0.000900 -0.000090   0.005239  0.004304  0.002776   
101  0.000981  0.001224   0.001918  0.002474   0.003855  0.002209  0.002480   
102 -0.000230  0.000468   0.000201  0.000450   0.001509  0.000411  0.000635   
103  0.000045  0.000091   0.001934  0.001499  -0.000474 -0.001516  0.000730   
104  0.000925  0.000516   0.000435  0.001084   0.000655  0.002671  0.001406   
105  0.000797  0.001177   0.000272  0.002062   0.004327  0.003678  0.002321   
106  0.001521  0.001226   0.001306  0.002018   0.003229  0.001852  0.002534   
107 -0.001806 -0.001317  -0.000244  0.001053   0.002443  0.003034  0.000174   
108 -0.000885 -0.000563   0.001216  0.002139   0.005137  0.003229  0.001436   
109 -0.000941 -0.000851  -0.000499  0.000937   0.000845  0.001037 -0.000026   
110  0.001723  0.001655   0.003054  0.003520   0.005550  0.004064  0.003261   
111  0.002251  0.002302   0.004914  0.005360  -0.001859 -0.000482  0.001819   
112  0.000181  0.000553  -0.000906  0.000885   0.001158  0.000628  0.000447   
113  0.001385  0.001198   0.006216  0.006221   0.002289  0.000733  0.003321   
114  0.001604  0.000762   0.003997  0.004547   0.003398  0.003518  0.004576   
115  0.000553  0.000282   0.000474  0.000387   0.001621  0.001733  0.002017   
116 -0.001210 -0.000969   0.008766  0.007398   0.010234  0.010694  0.006736   
117  0.000456  0.000941   0.004504  0.003852   0.012154  0.009648  0.009651   
119 -0.000935  0.000340   0.000098  0.001381   0.001315  0.001908  0.000251   
120  0.000292  0.000248   0.001190  0.001027   0.002675  0.002173  0.001138   
121 -0.000966 -0.000175  -0.000144 -0.000308   0.002405  0.000623 -0.000149   
122  0.000991  0.000625  -0.001336 -0.000771   0.000242  0.000364  0.000197   
123  0.000064 -0.000098  -0.000519 -0.000907   0.000170  0.001126  0.000346   
124  0.000647  0.000196   0.002939  0.002620   0.000507  0.002276  0.001337   
125 -0.002143 -0.000061   0.001643  0.000796  -0.001158  0.000336 -0.000649   
126  0.000818  0.002411   0.001549 -0.000051   0.001714  0.000733  0.001831   
127  0.000008  0.000005   0.001655 -0.000468   0.001210  0.000648  0.000995   
128 -0.000831 -0.000488  -0.000249 -0.000830   0.000606  0.000374  0.000184   
129  0.001672  0.002364   0.004298  0.005814   0.000891  0.002046  0.003620   
130  0.001241  0.001616   0.002382  0.001709   0.003413  0.002478  0.002205   
131  0.008149  0.008891   0.009420  0.010783   0.007142  0.006467  0.010948   
132 -0.000368  0.000487  -0.000998 -0.001622   0.001389  0.001530 -0.000237   
133  0.001800  0.000537   0.002941  0.002028   0.006933  0.006543  0.004638   
135  0.002171  0.002018   0.001917 -0.000077   0.005824  0.005809  0.003305   
136  0.001444  0.000112   0.003574  0.003717   0.001673  0.000563  0.003870   
137  0.001606  0.001386   0.000481  0.001449   0.002008  0.002060  0.004515   
138  0.001147  0.001655   0.000214  0.002427   0.002016  0.000665  0.002172   

              l_hip_ant            ...     l_hip           r_hip_ant  \
       across    within    across  ...    within    across    within   
100  0.001967  0.002376  0.001456  ...  0.003271  0.002123  0.003070   
101  0.001998  0.002243  0.001666  ...  0.003090  0.002397  0.000427   
102  0.000908 -0.000188  0.000178  ...  0.000702  0.000565  0.000173   
103  0.000373  0.000031 -0.000084  ...  0.000684  0.000277 -0.000517   
104  0.001508  0.000201 -0.000258  ...  0.000038 -0.000066  0.001121   
105  0.003301  0.001988  0.002520  ...  0.001882  0.002917 -0.000436   
106  0.002204  0.003200  0.003246  ...  0.003174  0.002888  0.000050   
107  0.000996 -0.000825 -0.001704  ... -0.000023  0.000368 -0.002597   
108  0.001337  0.000198 -0.000480  ...  0.001781  0.002110 -0.002924   
109  0.000742 -0.002060 -0.001715  ... -0.000157  0.000911  0.000006   
110  0.002903  0.002640  0.001754  ...  0.001383  0.000696

In [17]:
roi_mean_subj_sim_df.to_csv('singletrial_structured_roi_WithinAcross_Subject.csv', index=True)  